In [1]:
#数据预处理
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
import catboost as cab
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,precision_recall_fscore_support,roc_curve,auc,roc_auc_score
from sklearn.model_selection import GridSearchCV

F:\anaconda\envs\tensorflow\lib\site-packages\xgboost\__init__.py:29: FutureWarning: Python 3.5 support is deprecated; XGBoost will require Python 3.6+ in the near future. Consider upgrading to Python 3.6+.
  FutureWarning)


In [2]:
#本次示例中使用了训练集共24644个数据
with open(r'data/train_datafs.csv') as f:
    df_new = pd.read_csv(f,parse_dates=[0],index_col=0)
# with open(r'data/test_datafs.csv') as f:
#     df_test = pd.read_csv(f,parse_dates=[0],index_col=0)
#df_test = df_test.reset_index()
#特征选择
# select_feature=['industryco',
#  'regcap',
#  'reccap',
#  'industryphy',
#  'regcap_reccap',
#  'TAX_AMOUNT',
#  'enttypegb_enttypeitem',
#  'industryphy_enttypeitem',
#  'tfidi_opscope',
#  'enttypegb',
#  'gap_year',
#  'enttypegb_industryphy',
#  'enttypeminu',
#  'industryphy_industryco',
#  'enttypeitem',
#  'townsign',
#  'bucket_regcap',
#  'bgxmdm',
#  'empnum',
#  'positive_negtive',
#  'bucket_regcap_reccap',
#  'exenum',
#  'FORINVESTSIGN',
#  'oplocdistrict',
#  'industryco_enttypeitem',
#  'enttypegb_enttypeitem_industryphy_industryco',
#  'COLGRANUM',
#  'bucket_reccap',
#  'patent_num',
#  'EMPNUM',
# 'id']
select_feature2=['industryco',
 'regcap',
 'reccap',
 'industryphy',
 'regcap_reccap',
 'TAX_AMOUNT',
 'enttypegb_enttypeitem',
 'industryphy_enttypeitem',
 'tfidi_opscope',
 'enttypegb',
 'gap_year',
 'enttypegb_industryphy',
 'enttypeminu',
 'industryphy_industryco',
 'enttypeitem',
 'townsign',
 'bucket_regcap',
 'bgxmdm',
 'empnum',
 'positive_negtive',
 'bucket_regcap_reccap',
 'exenum',
 'FORINVESTSIGN',
 'oplocdistrict',
 'industryco_enttypeitem',
 'enttypegb_enttypeitem_industryphy_industryco',
 'COLGRANUM',
 'bucket_reccap',
 'patent_num',
 'EMPNUM',
 'label']

df_new=df_new[[i for i in df_new.columns if i in select_feature2]]
#df_test=df_test[[i for i in df_test.columns if i in select_feature]]
#处理'FORINVESTSIGN'
df_1= df_new['FORINVESTSIGN'].copy()
#df_2= df_test['FORINVESTSIGN'].copy()
#与
#m1 = df['FORINVESTSIGN'][(df_ != -1) & (df_ != 2)]

#或
df_new = df_new[(df_1 == -1) ^ (df_1 == 2)]
xtrain = df_new.iloc[:,:-1]
#ytrain = df_new.iloc[:,-1]
#test = df_test.iloc[:,1:]

#df = pd.concat([xtrain, test],axis=0)
#df.index = range(24644)
#分为类别特征和数值特征
cat_features=['industryco',
 'industryphy',
 'regcap_reccap',
 'enttypegb_enttypeitem',
 'industryphy_enttypeitem',
 'enttypegb',
 'enttypegb_industryphy',
 'enttypeminu',
 'industryphy_industryco',
 'enttypeitem',
 'townsign',
 'bucket_regcap',
 'positive_negtive',
 'bucket_regcap_reccap',
 'FORINVESTSIGN',
 'oplocdistrict',
 'industryco_enttypeitem',
 'enttypegb_enttypeitem_industryphy_industryco',
 'bucket_reccap',
 'patent_num']
df_cat = df_new[[i for i in df_new.columns if i in cat_features]]
num_feat = [ 'regcap',
            'reccap', 
            'TAX_AMOUNT',
            'tfidi_opscope',
            'gap_year',
            'bgxmdm',
            'empnum',
            'exenum',
             'COLGRANUM',
            'EMPNUM']
df_num=df_new[[i for i in df_new.columns if i in num_feat]]
#将每个类别特征编码改为1~len(该特征维数)，缺失值填0
def m(df):
    for i in cat_features:
#    print(i)
        labels = df[i].unique().tolist()
        if -1 in labels:
            labels.remove(-1)
            q = df[i][df[i] != -1]
            q = q.apply(lambda x:labels.index(x)+1)
            df[i][df[i] != -1] = q
        else:
            df[i] = df[i].apply(lambda x:labels.index(x)+1)
m(df_cat)
min_cols = df_num.min()
max_cols = df_num.max()
df_num = (df_num - min_cols) / (max_cols - min_cols)
df_1 = pd.concat([df_cat, df_num],axis=1)
xtrain = df_1.iloc[:14644,:]

F:\anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
F:\anaconda\envs\tensorflow\lib\site-packages\pandas\core\generic.py:9114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
F:\anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
F:\anaconda\envs\tensorflow\lib\site-packages\ipykernel_

In [3]:
ytrain = df_new.iloc[:,-1]
ytrain

id
47645761dc56bb8c5fae00114b768b5d9b6e917c3aec07c4    0
59b38c56de3836838082cfcb1a298951abfe15e6940c49ba    0
e9f7b28ec10e047000d16ab79e1b5e6da434a1697cce7818    0
f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb    0
9c7fa510616a6830b878f3c8c4317d93e1b022e7f22ae231    0
                                                   ..
59b38c56de3836837421cc5322074e10bb13a74257eb1969    1
f000950527a6feb6a532f0b773f795bb428e8646e87448e7    0
59b38c56de3836835d73a7fa9a782082ade3433adf3aaebc    0
f000950527a6feb62bf5fe41b57d6b3f599e32dd8c68f172    0
216bd2aaf4d07924009c6a8241b306e5a6050602204ebbbf    0
Name: label, Length: 14644, dtype: int64

In [4]:
#划分训练集和测试集
from sklearn.model_selection import train_test_split
dfTrain, dfTest, ytrain, ytest = train_test_split(xtrain,ytrain,test_size=0.3)#0.3是常用的划分,随机划分后数据的索引乱了
for i in [dfTrain, dfTest, ytrain, ytest]:
    i.index = range(i.shape[0])#索引换为0-shape[0]

In [10]:
#deepfm网络处理缺失值的方法
def preprocess(df):
    cols = list(df.columns)#[c for c in df.columns if c not in ["id", "target"]]
    df["missing_feat"] = np.sum((df[cols] == -1).values, axis=1)
    return df
dfTrain = preprocess(dfTrain)
dfTest = preprocess(dfTest)

F:\anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
df = pd.concat([dfTrain,dfTest],axis=0)

In [12]:
df.index = range(df.shape[0])

In [13]:
NUMERIC_COLS = num_feat
IGNORE_COLS = [
     "label"
]

In [14]:
# feature_dict = {}
# total_feature = 0
# for col in df.columns:
#     if col in IGNORE_COLS:
#         continue
#     elif col in NUMERIC_COLS:
#         feature_dict[col] = total_feature
#         total_feature += 1
#     else:
#         unique_val = df[col].unique()
#         feature_dict[col] = dict(zip(unique_val,range(total_feature,len(unique_val) + total_feature)))
#         total_feature += len(unique_val)
# print(total_feature)
# print(feature_dict)
feature_dict = {}
total_feature = 0
for col in df.columns:
    if col in IGNORE_COLS:
        continue
    elif col in NUMERIC_COLS:
        feature_dict[col] = total_feature
        total_feature += 1
    else:
        unique_val = df[col].unique()
        feature_dict[col] = dict(zip(unique_val,range(total_feature,len(unique_val) + total_feature)))
        total_feature += len(unique_val)
print(total_feature)
print(feature_dict)

3486
{'EMPNUM': 3476, 'bucket_reccap': {1: 1117, 2: 1118, 3: 1119}, 'enttypegb_enttypeitem_industryphy_industryco': {1: 2658, 2: 2768, 3: 2933, 4: 2577, 5: 2576, 6: 2721, 7: 2556, 8: 2565, 9: 2553, 10: 2531, 11: 2750, 12: 2799, 13: 2523, 14: 2614, 15: 2788, 16: 2528, 17: 2584, 18: 2978, 19: 2701, 20: 2542, 21: 2769, 22: 2555, 23: 2529, 24: 2626, 25: 2641, 26: 2535, 27: 2558, 28: 2994, 29: 2650, 30: 2591, 31: 2549, 32: 2599, 33: 2603, 34: 2539, 35: 2550, 36: 2589, 37: 2612, 38: 2536, 39: 3279, 40: 2613, 41: 2643, 42: 2537, 43: 2547, 44: 3060, 45: 3013, 46: 2684, 47: 3311, 48: 2582, 49: 2849, 50: 2600, 51: 2530, 52: 2947, 53: 2826, 54: 3265, 55: 2836, 56: 2822, 57: 2532, 58: 2861, 59: 2573, 60: 2860, 61: 2557, 62: 2534, 63: 2651, 64: 2562, 65: 2635, 66: 3141, 67: 2832, 68: 2545, 69: 2581, 70: 2611, 71: 2853, 72: 2571, 73: 2691, 74: 2588, 75: 3161, 76: 2708, 77: 2740, 78: 2566, 79: 2692, 80: 3074, 81: 2617, 82: 2649, 83: 2752, 84: 2569, 85: 2618, 86: 2655, 87: 2593, 88: 2559, 89: 3348, 90

In [15]:
import tensorflow as tf
from sklearn.metrics import roc_auc_score
"""模型参数"""
# params
dfm_params = {
    "use_fm": True,
    "use_deep": True,
    "embedding_size": 8,
    "dropout_fm": [1.0, 1.0],
    "deep_layers": [32, 32],
    "dropout_deep": [0.5, 0.5, 0.5],
    "deep_layers_activation": tf.nn.relu,
    "epoch": 30,
    "batch_size": 1024,
    "learning_rate": 0.001,
    "optimizer_type": "adam",
    "batch_norm": 1,
    "batch_norm_decay": 0.995,
    "l2_reg": 0.01,
    "verbose": True,
    "eval_metric": roc_auc_score,
    "random_seed": 2017
}
dfm_params['feature_size'] = total_feature
dfm_params['field_size'] = len(df.columns)

F:\anaconda\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
F:\anaconda\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
F:\anaconda\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
F:\anaconda\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWa

In [4]:
from DeepFM import DeepFM
# init a DeepFM model
dfm = DeepFM(**dfm_params)

# # fit a DeepFM model
# dfm.fit(Xi, Xv, ytrain)

In [6]:
from sklearn.ensemble import RandomForestClassifier

In [15]:
rf = RandomForestClassifier(oob_score=True, random_state=2020,
            n_estimators= 70,max_depth=13,min_samples_split=5,class_weight="balanced")

In [5]:
kind = ytrain
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,precision_recall_fscore_support,roc_curve,auc,roc_auc_score
def eval_score(y_test,y_pre):
    _,_,f_class,_=precision_recall_fscore_support(y_true=y_test,y_pred=y_pre,labels=[0,1],average=None)
    fper_class={'合法':f_class[0],'违法':f_class[1],'f1':f1_score(y_test,y_pre)}
    return fper_class

In [1]:
#随机森林组合deepfm
# mean_f1=0
# mean_f1Train=0
# n_splits=5
# sk = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=2020)
# answers = []
# for train, test in sk.split(dfTrain, kind):
#     x_train = dfTrain.iloc[train]
#     y_train = kind.iloc[train]
#     x_test = dfTrain.iloc[test]
#     y_test = kind.iloc[test]
    
#     #y_train = pd.get_dummies(y_train)
#     #y_test = pd.get_dummies(y_test)

#     """
#     对训练集进行转化
#     """
#     y_train = np.array(y_train).reshape(-1,).tolist()
#     #y_train = (np.array(y_train).reshape(-1,2)).astype(int).tolist()

#     train_feature_index = x_train.copy()
#     train_feature_value = x_train.copy()

#     for col in train_feature_index.columns:
#         if col in IGNORE_COLS:
#             train_feature_index.drop(col,axis=1,inplace=True)
#             train_feature_value.drop(col,axis=1,inplace=True)
#             continue
#         elif col in NUMERIC_COLS:
#             train_feature_index[col] = feature_dict[col]
#         else:
#             train_feature_index[col] = train_feature_index[col].map(feature_dict[col])
#             train_feature_value[col] = 1

#     # list of list of feature indices of each sample in the dataset
#     Xi = train_feature_index.values.tolist()
#     # list of list of feature values of each sample in the dataset
#     Xv = train_feature_value.values.tolist()

#     """
#     对验证集进行转化
#     """
#     y_test = np.array(y_test).reshape(-1,).tolist()
#     test_feature_index = x_test.copy()
#     test_feature_value = x_test.copy()

#     for col in test_feature_index.columns:
#         if col in IGNORE_COLS:
#             test_feature_index.drop(col,axis=1,inplace=True)
#             test_feature_value.drop(col,axis=1,inplace=True)
#             continue
#         elif col in NUMERIC_COLS:
#             test_feature_index[col] = feature_dict[col]
#         else:
#             test_feature_index[col] = test_feature_index[col].map(feature_dict[col])
#             test_feature_value[col] = 1
#     Xi_valid = test_feature_index.values.tolist()
#     Xv_valid = test_feature_value.values.tolist()

#     """
#     对测试集进行转化
#     """

#     test_feature_index1 = dfTest.copy()
#     test_feature_value1 = dfTest.copy()
#     test_feature_index1

#     for col in df.columns:
#         if col in IGNORE_COLS:
#             test_feature_index1.drop(col,axis=1,inplace=True)
#             test_feature_value1.drop(col,axis=1,inplace=True)
#             continue
#         elif col in NUMERIC_COLS:
#             test_feature_index1[col] = feature_dict[col]
#         else:
#             test_feature_index1[col] = test_feature_index1[col].map(feature_dict[col])
#             test_feature_value1[col] = 1
#     # list of list of feature indices of each sample in the dataset
#     Xi_test = test_feature_index1.values.tolist()
#     Xv_test = test_feature_value1.values.tolist()

#     dfm.fit(Xi, Xv, y_train, Xi_valid, Xv_valid, y_test, early_stopping=True)
#     dfinput = pd.DataFrame(dfm.out_concat_input(Xi,Xv))
#     dfinput_v = pd.DataFrame(dfm.out_concat_input(Xi_valid,Xv_valid))
#     dfinput_t = pd.DataFrame(dfm.out_concat_input(Xi_test,Xv_test))
# #     pred = dfm.predict(Xi_valid, Xv_valid)
# #     fper_class =  eval_score(yvalid, pred)
# #     mean_f1+=fper_class['f1']/n_splits
#     #print(fper_class)
#     #mean_dfinput += dfinput/n_splits
#     rf.fit(dfinput, y_train)
#     pred = rf.predict(dfinput_v)
#     fper_class =  eval_score(y_test, pred)
#     mean_f1+=fper_class['f1']/n_splits
    
#     test_rf = rf.predict_proba(dfinput_t)
#     answers.append(test_rf)

# fina=sum(answers)/n_splits#
# #fina=np.sqrt(sum(np.array(answers)**2)/n_splits)#平方平均
# #fina=fina[:,1]
# #print('mean valf1:',mean_f1)
# #print('mean trainf1:',mean_f1Train)
# print(mean_f1)
# print(fina.shape)

In [17]:
# #单独使用deepfm结果
# fina2 = dfm.predict(Xi_test,Xv_test)
# m =[]
# for i in fina2:
#     if i>=0.5:
#         m.append(1)
#     else:
#         m.append(0)
# ytest
# eval_score(ytest, m)

{'f1': 0.8045977011494253, '合法': 0.9876602951850955, '违法': 0.8045977011494253}

In [18]:
# #随机森林组合deepfm
# m1 =[]
# for i in fina[:,1]:
#     if i>=0.5:
#         m1.append(1)
#     else:
#         m1.append(0)
# eval_score(ytest,m1)

{'f1': 0.8120300751879698, '合法': 0.9878875968992248, '违法': 0.8120300751879698}

In [20]:
# #单独使用随机森林
# kind=ytrain
# mean_f1=0
# mean_f1Train=0
# n_splits=5
# sk1 = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=2020)
# answers = []
# for train, test in sk1.split(dfTrain, kind):
#     x_train = dfTrain.iloc[train]
#     y_train = kind.iloc[train]
#     x_test = dfTrain.iloc[test]
#     y_test = kind.iloc[test]
    
#     rf.fit(x_train, y_train)
#     pred = rf.predict(x_test)
#     fper_class =  eval_score(y_test, pred)
#     mean_f1+=fper_class['f1']/n_splits
    
#     test_rf = rf.predict_proba(dfTest)
#     answers.append(test_rf)

# fina=sum(answers)/n_splits#
# #fina=np.sqrt(sum(np.array(answers)**2)/n_splits)#平方平均
# #fina=fina[:,1]
# #print('mean valf1:',mean_f1)
# #print('mean trainf1:',mean_f1Train)
# print(mean_f1)
# print(fina.shape)

0.7583699169200536
(4394, 2)


In [21]:
# #单独使用随机森林
# m2 =[]
# for i in fina[:,1]:
#     if i>=0.5:
#         m2.append(1)
#     else:
#         m2.append(0)
# eval_score(ytest,m2)  

{'f1': 0.7887323943661972, '合法': 0.9854014598540147, '违法': 0.7887323943661972}

In [30]:
#使用树模型融合
xlf=xgb.XGBClassifier(max_depth=7
                      ,learning_rate=0.05
                      ,n_estimators=53
                      ,reg_alpha=0.005
                      ,n_jobs=8
                      ,importance_type='total_cover'
                     ,is_unbalance=True)

llf=lgb.LGBMClassifier(num_leaves=9
                           ,max_depth=5
                           ,learning_rate=0.05
                           ,n_estimators=80
                           ,n_jobs=8
                           ,is_unbalance=True)
class_weights = np.array([1., 14.])
class_weights = class_weights.tolist()
print(class_weights)
clf=cab.CatBoostClassifier(iterations=60
                             ,learning_rate=0.05
                             ,depth=10
                             ,silent=True
                             ,thread_count=8
                             ,task_type='CPU'
                             ,cat_features=None
                             ,class_weights=class_weights)

rf = RandomForestClassifier(oob_score=True, random_state=2020,
            n_estimators= 70,max_depth=13,min_samples_split=5,class_weight="balanced")

[1.0, 14.0]


In [31]:
#模型融合，最终方法
details = []
answers = []
mean_f1=0
n_splits=5
sk = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=2020)
cnt=0
for train, test in sk.split(dfTrain, kind):
    x_train = dfTrain.iloc[train]
    y_train = kind.iloc[train]
    x_test = dfTrain.iloc[test]
    y_test = kind.iloc[test]

    xlf.fit(x_train, y_train)
    pred_xgb = xlf.predict(x_test)
    weight_xgb = eval_score(y_test,pred_xgb)['f1']

    llf.fit(x_train, y_train)
    pred_llf = llf.predict(x_test)
    weight_lgb = eval_score(y_test,pred_llf)['f1']

    clf.fit(x_train, y_train)
    pred_cab = clf.predict(x_test)
    weight_cab =  eval_score(y_test,pred_cab)['f1']

    rf.fit(x_train, y_train)
    pred_rf = rf.predict(x_test)
    weight_rf =  eval_score(y_test,pred_rf)['f1']


    prob_xgb = xlf.predict_proba(x_test)
    prob_lgb = llf.predict_proba(x_test)
    prob_cab = clf.predict_proba(x_test)
    prob_rf = rf.predict_proba(x_test)

    scores = []
    ijkl = []
    weight = np.arange(0, 1.05, 0.1)
    for i, item1 in enumerate(weight):
        for j, item2 in enumerate(weight[weight <= (1 - item1)]):
            for k, item3 in enumerate(weight[weight <= (1 - item1-item2)]):
                prob_end = prob_xgb * item1 + prob_lgb * item2 + prob_cab *item3+prob_rf*(1 - item1 - item2-item3)
                #prob_end = np.sqrt(prob_xgb**2 * item1 + prob_lgb**2 * item2 + prob_cab**2 *item3+prob_rf**2*(1 - item1 - item2-item3))
                score = eval_score(y_test,np.argmax(prob_end,axis=1))['f1']
                scores.append(score)
                ijkl.append((item1, item2,item3, 1 - item1 - item2-item3))

    ii = ijkl[np.argmax(scores)][0]
    jj = ijkl[np.argmax(scores)][1]
    kk = ijkl[np.argmax(scores)][2]
    ll = ijkl[np.argmax(scores)][3]

    details.append(max(scores))
    details.append(weight_xgb)
    details.append(weight_lgb)
    details.append(weight_cab)
    details.append(weight_rf)
    details.append(ii)
    details.append(jj)
    details.append(kk)
    details.append(ll)

    cnt+=1
    print('每{}次验证的f1:{}'.format(cnt,max(scores)))
    mean_f1+=max(scores)/n_splits

    test_xgb = xlf.predict_proba(dfTest)
    test_lgb = llf.predict_proba(dfTest)
    test_cab = clf.predict_proba(dfTest)
    test_rf = rf.predict_proba(dfTest)
    #加权平均
    ans = test_xgb * ii + test_lgb * jj + test_cab * kk + test_rf*ll#加权平均
    #加权平方平均
    #ans = np.sqrt(test_xgb**2 * ii + test_lgb**2 * jj + test_cab**2 * kk + test_rf**2*ll)
    answers.append(ans)
print('mean f1:',mean_f1)

[12:22:24] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { is_unbalance } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


每1次验证的f1:0.7906976744186047
[12:22:30] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { is_unbalance } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


每2次验证的f1:0.8016528925619835
[12:22:35] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { is_unbalance } might not be used.

  This may no

In [32]:
#模型融合在测试集上的效果
fina=sum(answers)/n_splits
m3 =[]
for i in fina[:,1]:
    if i>=0.5:
        m3.append(1)
    else:
        m3.append(0)
eval_score(ytest,m3) 

{'f1': 0.8451242829827916, '合法': 0.9901996370235935, '违法': 0.8451242829827916}

In [33]:
#使用deepfm与树模型融合
mean_f1=0
mean_f1Train=0
n_splits=5
sk = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=2020)
answers = []
for train, test in sk.split(dfTrain, kind):
    x_train = dfTrain.iloc[train]
    y_train = kind.iloc[train]
    x_test = dfTrain.iloc[test]
    y_test = kind.iloc[test]
    
    #y_train = pd.get_dummies(y_train)
    #y_test = pd.get_dummies(y_test)

    """
    对训练集进行转化
    """
    y_train = np.array(y_train).reshape(-1,).tolist()
    #y_train = (np.array(y_train).reshape(-1,2)).astype(int).tolist()

    train_feature_index = x_train.copy()
    train_feature_value = x_train.copy()

    for col in train_feature_index.columns:
        if col in IGNORE_COLS:
            train_feature_index.drop(col,axis=1,inplace=True)
            train_feature_value.drop(col,axis=1,inplace=True)
            continue
        elif col in NUMERIC_COLS:
            train_feature_index[col] = feature_dict[col]
        else:
            train_feature_index[col] = train_feature_index[col].map(feature_dict[col])
            train_feature_value[col] = 1

    # list of list of feature indices of each sample in the dataset
    Xi = train_feature_index.values.tolist()
    # list of list of feature values of each sample in the dataset
    Xv = train_feature_value.values.tolist()

    """
    对验证集进行转化
    """
    y_test = np.array(y_test).reshape(-1,).tolist()
    test_feature_index = x_test.copy()
    test_feature_value = x_test.copy()

    for col in test_feature_index.columns:
        if col in IGNORE_COLS:
            test_feature_index.drop(col,axis=1,inplace=True)
            test_feature_value.drop(col,axis=1,inplace=True)
            continue
        elif col in NUMERIC_COLS:
            test_feature_index[col] = feature_dict[col]
        else:
            test_feature_index[col] = test_feature_index[col].map(feature_dict[col])
            test_feature_value[col] = 1
    Xi_valid = test_feature_index.values.tolist()
    Xv_valid = test_feature_value.values.tolist()

    """
    对测试集进行转化
    """

    test_feature_index1 = dfTest.copy()
    test_feature_value1 = dfTest.copy()
    test_feature_index1

    for col in df.columns:
        if col in IGNORE_COLS:
            test_feature_index1.drop(col,axis=1,inplace=True)
            test_feature_value1.drop(col,axis=1,inplace=True)
            continue
        elif col in NUMERIC_COLS:
            test_feature_index1[col] = feature_dict[col]
        else:
            test_feature_index1[col] = test_feature_index1[col].map(feature_dict[col])
            test_feature_value1[col] = 1
    # list of list of feature indices of each sample in the dataset
    Xi_test = test_feature_index1.values.tolist()
    Xv_test = test_feature_value1.values.tolist()

    dfm.fit(Xi, Xv, y_train, Xi_valid, Xv_valid, y_test, early_stopping=True)
    dfinput = pd.DataFrame(dfm.out_concat_input(Xi,Xv))
    dfinput_v = pd.DataFrame(dfm.out_concat_input(Xi_valid,Xv_valid))
    dfinput_t = pd.DataFrame(dfm.out_concat_input(Xi_test,Xv_test))
#     pred = dfm.predict(Xi_valid, Xv_valid)
#     fper_class =  eval_score(yvalid, pred)
#     mean_f1+=fper_class['f1']/n_splits
    #print(fper_class)
    #mean_dfinput += dfinput/n_splits
    xlf.fit(dfinput, y_train)
    pred_xgb = xlf.predict(dfinput_v)
    weight_xgb = eval_score(y_test,pred_xgb)['f1']

    llf.fit(dfinput, y_train)
    pred_llf = llf.predict(dfinput_v)
    weight_lgb = eval_score(y_test,pred_llf)['f1']

    clf.fit(dfinput, y_train)
    pred_cab = clf.predict(dfinput_v)
    weight_cab =  eval_score(y_test,pred_cab)['f1']

    rf.fit(dfinput, y_train)
    pred_rf = rf.predict(dfinput_v)
    weight_rf =  eval_score(y_test,pred_rf)['f1']


    prob_xgb = xlf.predict_proba(dfinput_v)
    prob_lgb = llf.predict_proba(dfinput_v)
    prob_cab = clf.predict_proba(dfinput_v)
    prob_rf = rf.predict_proba(dfinput_v)

    scores = []
    ijkl = []
    weight = np.arange(0, 1.05, 0.1)
    for i, item1 in enumerate(weight):
        for j, item2 in enumerate(weight[weight <= (1 - item1)]):
            for k, item3 in enumerate(weight[weight <= (1 - item1-item2)]):
                prob_end = prob_xgb * item1 + prob_lgb * item2 + prob_cab *item3+prob_rf*(1 - item1 - item2-item3)
                #prob_end = np.sqrt(prob_xgb**2 * item1 + prob_lgb**2 * item2 + prob_cab**2 *item3+prob_rf**2*(1 - item1 - item2-item3))
                score = eval_score(y_test,np.argmax(prob_end,axis=1))['f1']
                scores.append(score)
                ijkl.append((item1, item2,item3, 1 - item1 - item2-item3))

    ii = ijkl[np.argmax(scores)][0]
    jj = ijkl[np.argmax(scores)][1]
    kk = ijkl[np.argmax(scores)][2]
    ll = ijkl[np.argmax(scores)][3]

    details.append(max(scores))
    details.append(weight_xgb)
    details.append(weight_lgb)
    details.append(weight_cab)
    details.append(weight_rf)
    details.append(ii)
    details.append(jj)
    details.append(kk)
    details.append(ll)

    cnt+=1
    print('每{}次验证的f1:{}'.format(cnt,max(scores)))
    mean_f1+=max(scores)/n_splits

    test_xgb = xlf.predict_proba(dfinput_t)
    test_lgb = llf.predict_proba(dfinput_t)
    test_cab = clf.predict_proba(dfinput_t)
    test_rf = rf.predict_proba(dfinput_t)
    #加权平均
    ans = test_xgb * ii + test_lgb * jj + test_cab * kk + test_rf*ll#加权平均
    #加权平方平均
    #ans = np.sqrt(test_xgb**2 * ii + test_lgb**2 * jj + test_cab**2 * kk + test_rf**2*ll)
    answers.append(ans)
print('mean f1:',mean_f1)

[1] train-result=0.9972, valid-result=0.9981 [0.3 s]
[2] train-result=0.9975, valid-result=0.9979 [0.3 s]
[3] train-result=0.9973, valid-result=0.9979 [0.3 s]
[4] train-result=0.9974, valid-result=0.9978 [0.3 s]
[5] train-result=0.9973, valid-result=0.9977 [0.3 s]
[12:23:15] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { is_unbalance } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


每6次验证的f1:0.8879999999999999
[1] train-result=0.9964, valid-result=0.9980 [0.5 s]
[2] train-result=0.9971, valid-result=0.9980 [0.4 s]
[3] train-result=0.9962, valid-result=0.9981 [0.5 s]
[4] train-result=0.9967, valid-result=0.9980 [0.4 s]
[5] train-result=0.9971, valid-result=0.9978 [0.4 s]
[6] train-result=0.9966, valid-result=0.9977 [0.5 s]

In [34]:
#单独使用deepfm结果
fina2 = dfm.predict(Xi_test,Xv_test)
m =[]
for i in fina2:
    if i>=0.5:
        m.append(1)
    else:
        m.append(0)
ytest
eval_score(ytest, m)

{'f1': 0.7984344422700587, '合法': 0.9875558777334783, '违法': 0.7984344422700587}

In [35]:
#使用deepfm与树模型融合组合的结果，验证集高，但测试集低
fina=sum(answers)/n_splits
m4 =[]
for i in fina[:,1]:
    if i>=0.5:
        m4.append(1)
    else:
        m4.append(0)
eval_score(ytest,m4)

{'f1': 0.7939508506616257, '合法': 0.9868022763046373, '违法': 0.7939508506616257}

In [148]:
# pd.DataFrame({"id": Test1['id'], "score": y_pred.flatten()}).to_csv(
#         "data/pred_result.csv", index=False, float_format="%.5f")